### This script will combine the cards.feather and decks.feather into one dataframe which can be used for input to a model

In [186]:
import pandas as pd
import numpy as np
import os
import logging
from IPython.display import display, HTML


In [187]:
# Define a custom log handler that writes messages to the notebook output
class NotebookLogHandler(logging.Handler):
    def emit(self, record):
        message = self.format(record)
        display(HTML(f'<p style="color: {record.levelname.lower()}">{message}</p>'))

# Create a logger and set its level to INFO
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a formatter and add it to the logger
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler = NotebookLogHandler()
handler.setFormatter(formatter)
logger.addHandler(handler)

#### Read in our cards and deck data

In [188]:
# Get the absolute path of the current working directory
current_dir = os.path.abspath(os.getcwd())

# Get the absolute path of the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Read our card database
file_path = os.path.join(parent_dir, 'data', 'mtg_card_database')

card_fp = os.path.join(parent_dir, 'data', 'cards.feather')
deck_fp = os.path.join(parent_dir, 'data', 'decks.feather')

card_df = pd.read_feather(card_fp)
deck_df = pd.read_feather(deck_fp)

In [189]:
test_deck = deck_df.iloc[2][0]
test_deck = pd.DataFrame(data={'card+amt': deck_df.iloc[2][0]})
test_deck

,card+amt
0,"3 Atraxa, Grand Unifier"
1,4 Fable of the Mirror-Breaker
2,2 Courier's Briefcase
3,4 Esika's Chariot
4,1 Tear Asunder
5,2 Sheoldred's Edict
6,4 Fatal Push
7,4 Thoughtseize
8,4 Transmogrify
9,4 Careful Cultivation


Extract the card and the card amount

In [190]:
def extract_card(row):
    try:
        card = row[1:].strip()
        return card
    except:
        return row

def extract_card_amt(row):
    try:
        amount_of_card = row[0]
        return amount_of_card
    except:
        return row
    

test_deck = test_deck.copy()
test_deck['card'] = test_deck['card+amt'].apply(lambda row: extract_card(row))
test_deck['amount'] = test_deck['card+amt'].apply(lambda row: extract_card_amt(row))

test_deck = test_deck.drop(columns='card+amt')

In [191]:
def handle_dfc(row):
    try:
        return row.split(' // ')[0]
    except:
        return row

card_df['name2'] = card_df['name'].apply(lambda row: handle_dfc(row))

In [192]:
merged_df = pd.merge(left=test_deck, right=card_df, left_on='card', right_on='name2', how='inner')

card_df[card_df['name'] == 'Fable of the Mirror-Breaker // Reflection of Kiki-Jiki'].name2

1832    Fable of the Mirror-Breaker
Name: name2, dtype: object

In [193]:
test_deck

,card,amount
0,"Atraxa, Grand Unifier",3
1,Fable of the Mirror-Breaker,4
2,Courier's Briefcase,2
3,Esika's Chariot,4
4,Tear Asunder,1
5,Sheoldred's Edict,2
6,Fatal Push,4
7,Thoughtseize,4
8,Transmogrify,4
9,Careful Cultivation,4


In [194]:
merged_df

,card,amount,name,cmc,type_line,cast_cost_W,cast_cost_U,cast_cost_B,cast_cost_R,cast_cost_G,...,produces_G,produces_C,produces_P,num_x_in_mana_cost,has_looting,has_carddraw,makes_treasure_tokens,reduced_spells,free_spells,name2
0,"Atraxa, Grand Unifier",3,"Atraxa, Grand Unifier",7.0,Legendary Creature — Phyrexian Angel,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,"Atraxa, Grand Unifier"
1,Fable of the Mirror-Breaker,4,Fable of the Mirror-Breaker // Reflection of K...,3.0,Enchantment — Saga // Enchantment Creature — G...,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,1,1,0,0,Fable of the Mirror-Breaker
2,Courier's Briefcase,2,Courier's Briefcase,2.0,Artifact — Treasure,0.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0,1,0,0,0,Courier's Briefcase
3,Esika's Chariot,4,Esika's Chariot,4.0,Legendary Artifact — Vehicle,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,Esika's Chariot
4,Tear Asunder,1,Tear Asunder,2.0,Instant,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,Tear Asunder
5,Sheoldred's Edict,2,Sheoldred's Edict,2.0,Instant,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Sheoldred's Edict
6,Fatal Push,4,Fatal Push,1.0,Instant,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Fatal Push
7,Thoughtseize,4,Thoughtseize,1.0,Sorcery,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Thoughtseize
8,Transmogrify,4,Transmogrify,4.0,Sorcery,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,Transmogrify
9,Careful Cultivation,4,Careful Cultivation,3.0,Enchantment — Aura,0.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,0,0,Careful Cultivation
